In [26]:
import pymysql as psq
import csv
import mysecrets

#Setting up the MySQL connection
conn = psq.connect(host = mysecrets.host, port = 3306, user = mysecrets.user, passwd = mysecrets.passwd, db = 'is426', autocommit = True)

cur = conn.cursor(psq.cursors.DictCursor)

#Clear all tables
command = '''DROP TABLE IF EXISTS `lichtemj_data`;'''
cur.execute(command)
command = '''DROP TABLE IF EXISTS `lichtemj_codes`;'''
cur.execute(command)
command = '''DROP TABLE IF EXISTS `lichtemj_indicators`'''
cur.execute(command)

command = '''CREATE TABLE `lichtemj_indicators`(
`id` int(3) NOT NULL AUTO_INCREMENT,
`Month` int(2) NOT NULL,
`Year` int(4) NOT NULL,
`logan_passengers` int(10) NOT NULL,
`hotel_avg_daily_rate` decimal(7, 2),
`unemp_rate` decimal(4, 3) NOT NULL,
PRIMARY KEY (`id`));'''
cur.execute(command)
command = '''CREATE TABLE `lichtemj_codes`(
`id` int(5) NOT NULL AUTO_INCREMENT,
`CODE` int(5) NOT NULL,
`NAME` VARCHAR(255) NOT NULL,
PRIMARY KEY (`id`));'''
cur.execute(command)
command = '''CREATE TABLE `lichtemj_data`(
`INCIDENT_NUMBER` int(20) NOT NULL AUTO_INCREMENT,
`OFFENSE_CODE` int(5) NOT NULL,
`YEAR` int(4) NOT NULL,
`MONTH` int(2) NOT NULL,
PRIMARY KEY (`INCIDENT_NUMBER`))'''
cur.execute(command)

#Load indicators data
with open('data/economic-indicators.csv') as f:
    data = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]
for row in data:
    sql_f = '''INSERT INTO `lichtemj_indicators`(`Month`, `Year`, `logan_passengers`, `hotel_avg_daily_rate`, `unemp_rate`)
    VALUES (%s, %s, %s, %s, %s)'''
    tokens_f = [row['Month'], row['Year'], row['logan_passengers'], row['hotel_avg_daily_rate'], row['unemp_rate']]
    cur.execute(sql_f, tokens_f)
f.close()

#Load offense codes
with open('data/rmsoffensecodes.csv') as g:
    codes = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(g, skipinitialspace=True)]
for row in codes:
    sql_g = '''INSERT INTO `lichtemj_codes`(`CODE`, `NAME`)
    VALUES (%s, %s)'''
    tokens_g = [row['ï»¿CODE'], row['NAME']]
    cur.execute(sql_g, tokens_g)
g.close()

#Load applicable offense data through a loop
i = 2015
for i in range (2015, 2019):
    h_name = f'data/crime-incident-reports-{i}.csv'
    with open(h_name) as h:
        log = [{k: str(v) for k, v in row.items()}
            for row in csv.DictReader(h, skipinitialspace=True)]
    for row in log:
        sql_h = '''INSERT INTO `lichtemj_data`(`OFFENSE_CODE`, `YEAR`, `MONTH`)
        VALUES (%s, %s, %s)'''
        tokens_h = [row['OFFENSE_CODE'], row['YEAR'], row['MONTH']]
        cur.execute(sql_h, tokens_h)
    h.close()
#Note: This cell is to show my methodology and should not be run again
#This cell took 1877 seconds to execute (AKA 31:17)
#Operational work will be in a new cell

OperationalError: (1054, "Unknown column 'd.OFFENSE_CODE_ID' in 'where clause'")

In [27]:
import pymysql as psq
import csv
import mysecrets

#Setting up the MySQL connection
conn = psq.connect(host = mysecrets.host, port = 3306, user = mysecrets.user, passwd = mysecrets.passwd, db = 'is426', autocommit = True)

cur = conn.cursor(psq.cursors.DictCursor)
#Aggregate and retrieve the data
sql = '''SELECT COUNT(d.Incident_Number), i.Year, i.Month, i.logan_passengers, i.hotel_avg_daily_rate, i.unemp_rate, d.Month, d.Year, d.OFFENSE_CODE
FROM lichtemj_indicators i, lichtemj_data d, lichtemj_codes c
WHERE i.Month = d.Month AND i.Year = d.Year AND d.OFFENSE_CODE = c.CODE
GROUP BY i.Month, i.Year, d.OFFENSE_CODE'''
cur.execute(sql)

#For some reason this takes too long to grab from MySQL in Jupyter so I just executed the code in the terminal
#AND IT WORKED PERFECTLY
#New cell for the visualization

6854

In [29]:
#Charts from downloaded file of monthly crime vs indicators occurences
#load into a dictionary

with open('data/crimevsindicators.csv') as x:
    comparisons = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(x, skipinitialspace=True)]

maxcount = 0
mincount = 1000000000000
codemax = ''
codemin = ''
timestampmax = ''
timestampmin = ''
for row in comparisons:
    if int(row['Count']) > maxcount:
        maxcount = int(row['Count'])
        codemax = row['Code']
        timestampmax = f"{row['Month']}/{row['Year']}"
    if int(row['Count']) < mincount:
        mincount = int(row['Count'])
        codemin = row['Code']
        timestampmin = f"{row['Month']}/{row['Year']}"
        
print(f'The highest occurence in a month was of code {codemax} at {maxcount} times during {timestampmax}. The lowest occurence type in a month was code {codemin}, with just {mincount} occurences during {timestampmin}.')
print("------------------------------")
#Search for occurences
targetcode = '3006'
totalcount = 0
print(f"Incident counts for code {targetcode}")
for row in comparisons:
    if row['Code'] == targetcode:
        timestamp = f"{row['Month']}/{row['Year']}"
        occurence_count = int(row['Count'])
        totalcount += occurence_count
        print(f"{timestamp}: {occurence_count} incidents")
print(f"Total incidents: {totalcount}")
print("--------------------------------")
#Compare Economic Indicators to Crime data
maxrate = 0
minrate = 1000000000000
targetcode = '301'
timestampmax = ''
timestampmin = ''
maxoc = ''
minoc = ''
for row in comparisons:
    if float(row['Avg Hotel Daily Rate']) > maxrate:
        if (row['Code']) == targetcode:
            maxrate = float(row['Avg Hotel Daily Rate'])
            maxoc = row['Count']
            timestampmax = f"{row['Month']}/{row['Year']}"
    if float(row['Avg Hotel Daily Rate']) < minrate:
        if row['Code'] == targetcode:
            minrate = float(row['Avg Hotel Daily Rate'])
            timestampmin = f"{row['Month']}/{row['Year']}"
            minoc = row['Count']
print(f"The maximum hotel average daily rate was {maxrate} on {timestampmax}, during which {maxoc} incidents of code {targetcode} occured. The minimum rate was {minrate} on {timestampmin}, with {minoc} incidents of code {targetcode}.")
#In the interest of time, I will not be going over everything else that can be done with this data, as there are a metric ton of possibilities

The highest occurence in a month was of code 3006 at 1390 times during 5/2018. The lowest occurence type in a month was code 561, with just 1 occurences during 1/2016.
------------------------------
Incident counts for code 3006
1/2016: 848 incidents
1/2017: 994 incidents
1/2018: 1006 incidents
2/2016: 824 incidents
2/2017: 948 incidents
2/2018: 972 incidents
3/2016: 942 incidents
3/2017: 1178 incidents
3/2018: 1060 incidents
4/2016: 888 incidents
4/2017: 1018 incidents
4/2018: 1236 incidents
5/2016: 1066 incidents
5/2017: 1218 incidents
5/2018: 1390 incidents
6/2015: 324 incidents
6/2016: 932 incidents
6/2017: 1114 incidents
6/2018: 1314 incidents
7/2015: 694 incidents
7/2016: 992 incidents
7/2017: 1022 incidents
7/2018: 1150 incidents
8/2015: 624 incidents
8/2016: 878 incidents
8/2017: 1038 incidents
8/2018: 1088 incidents
9/2015: 748 incidents
9/2016: 892 incidents
9/2017: 1100 incidents
9/2018: 1170 incidents
10/2015: 742 incidents
10/2016: 874 incidents
10/2017: 1032 incidents
10/

In [5]:
#Resolving key error in dictionary
with open('data/rmsoffensecodes.csv') as g:
    codes = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(g, skipinitialspace=True)]
print(codes[0].keys())


dict_keys(['ï»¿CODE', 'NAME'])
